In [1]:
from sage.algebras.flag_algebras import *

def test_identify(n, ftype_points, edges, C0, C1, C2):
    return colored_identify(2, [[0], [1, 2]], n, ftype_points, edges=edges, C0=C0, C1=C1, C2=C2)

def test_generate(n):
    return colored_generate(2, [[0], [1, 2]], n)

TT = CombinatorialTheory("12ColGraph", test_generate, test_identify, edges=2, C0=1, C1=1, C2=1)

In [2]:
# Exclude triangles 6.1.6 (1)
f = TT.generate_flags(3)
TT.exclude(f[-6:])

In [3]:
# Exclude disjoint edges 6.1.6 (3)
# The edges share a vertex
f = TT.generate_flags(3)
avoid = []
for ff in f:
    if len(ff.blocks()['edges']) != 2:
        continue
    yep = True
    for edge in ff.blocks()['edges']:
        for i in range(0, 3):
            if [edge[0]] in ff.blocks()['C'+str(i)] and [edge[1]] in ff.blocks()['C'+str(i)]:
                yep = False
    if not yep:
        continue
    if ff.blocks()['C0'] == [] or ff.blocks()['C1'] == [] or ff.blocks()['C2'] == []:
        continue
    avoid.append(ff)
TT.exclude(avoid)
# The edges are vertex disjoint
f = TT.generate_flags(4)
avoid = []
for ff in f:
    if len(ff.blocks()['edges']) != 2:
        continue
    yep = True
    for edge in ff.blocks()['edges']:
        for i in range(0, 3):
            if [edge[0]] in ff.blocks()['C'+str(i)] and [edge[1]] in ff.blocks()['C'+str(i)]:
                yep = False
    if not yep:
        continue
    if ff.blocks()['C0'] == [] or ff.blocks()['C1'] == [] or ff.blocks()['C2'] == []:
        continue
    if ff.blocks()['edges'] != [[0, 2], [1, 3]]:
        continue
    avoid.append(ff)
TT.exclude(avoid)

In [4]:
# Exclude special paths 6.1.6 (2)
f = TT.generate_flags(4)
avoid = []
for ff in f:
    if len(ff.blocks()['edges']) != 3:
        continue
    degree = [0, 0, 0, 0]
    for edge in ff.blocks()['edges']:
        for v in edge:
            degree[v] += 1
    if sorted(degree) != [1, 1, 2, 2]:
        continue
    if ff.blocks()['C0'] == [] or ff.blocks()['C1'] == [] or ff.blocks()['C2'] == []:
        continue
    avoid.append(ff)
TT.exclude(avoid)

In [5]:
# Assumptions
edge_12 = TT.generate_flags(2)[-1]
edge_01 = TT.generate_flags(2)[5]
positives = [edge_12 - edge_01]

In [6]:
# Missing edges (correct?)
M = 1 + edge_12 - 1

# Bad edges (correct?)
f = TT.generate_flags(2)[-4:-1]
B = 1
for ff in f:
    B += ff
B -= 1

In [7]:
# Optimize (correct?)
x = TT.optimize(B - M, 5, maximize=False, positives=positives)
print(x)

Base flags generated, their number is 1339
The relevant ftypes are constructed, their number is 34
Block sizes before symmetric/asymmetric change is applied: [26, 42, 16, 24, 24, 24, 24, 24, 16, 22, 22, 22, 18, 22, 18, 24, 22, 22, 16, 22, 22, 22, 18, 22, 24, 22, 18, 22, 16, 24, 24, 24, 24, 24]


Done with mult table for Ftype on 3 points with edges=[[0, 1], [0, 2], [1, 2]], C0=[], C1=[[0], [1]], C2=[[2]]: : 34it [00:01, 31.67it/s]  


Tables finished


Done with positivity constraint 0: 100%|██████████| 1/1 [00:00<00:00,  5.97it/s]


Constraints finished
Running sdp without construction. Used block sizes are [26, 42, 8, 8, 18, 6, 18, 6, 14, 10, 12, 12, 18, 6, 12, 4, 22, 17, 5, 22, 18, 17, 5, 11, 7, 18, 6, 22, 17, 5, 12, 4, 17, 5, 22, 22, 18, 17, 5, 18, 6, 17, 5, 11, 7, 22, 8, 8, 18, 6, 18, 6, 14, 10, 12, 12, 18, 6, -1339, -34]
CSDP 6.2.0
Iter:  0 Ap: 0.00e+00 Pobj:  0.0000000e+00 Ad: 0.00e+00 Dobj:  0.0000000e+00 
Iter:  1 Ap: 2.59e-01 Pobj: -1.5740740e+01 Ad: 2.46e-01 Dobj: -1.3371268e+02 
Iter:  2 Ap: 6.55e-01 Pobj: -6.4138718e+01 Ad: 4.12e-01 Dobj: -5.3448065e+01 
Iter:  3 Ap: 1.00e+00 Pobj: -1.2133757e+02 Ad: 5.58e-01 Dobj: -2.6598240e+01 
Iter:  4 Ap: 1.00e+00 Pobj: -1.3084414e+02 Ad: 8.97e-01 Dobj: -2.2637703e+00 
Iter:  5 Ap: 1.00e+00 Pobj: -1.3896838e+02 Ad: 9.23e-01 Dobj: -3.2835726e-02 
Iter:  6 Ap: 9.86e-01 Pobj: -1.4820542e+02 Ad: 8.03e-01 Dobj:  1.1533663e-01 
Iter:  7 Ap: 3.14e-01 Pobj: -1.5857884e+02 Ad: 7.03e-01 Dobj:  1.4331386e-01 
Iter:  8 Ap: 2.34e-01 Pobj: -1.6244804e+02 Ad: 7.05e-01 Dobj:  1.4